In [1]:
using NPZ
using LinearAlgebra

dataPath = "../../data/processed/sstReducedStateCOPERNICUS20102019Prepared.npz"
data = npzread(dataPath)
# data contient le résultat final du prétraitement

PCsTrain = data["PCsTrain"] # Coefficients temporels des modes EOF
PCsVal = data["PCsVal"] # PCsTrain sur période future
tTrain = data["tTrain"] # Vecteur temps de PCsTrain
tVal = data["tVal"] # Vecteur temps de PCsVal
std = data["std"] # Ecart-type de chaque mode PCA

150-element Vector{Float32}:
 56.026962
 13.532461
  9.163721
  6.821123
  5.764589
  5.054141
  4.6614556
  3.8260977
  3.32742
  3.2594903
  ⋮
  0.4649968
  0.4605963
  0.45858616
  0.44919217
  0.4533658
  0.45195308
  0.4485787
  0.44228557
  0.4421681

## Mise en forme des données

In [2]:
T = length(tTrain)
sizePCs = size(PCsTrain)
println("PCsTrain size = ", sizePCs," and length(tTrain) = ", T)

zTrain = nothing 

if (sizePCs[1] == T) 
    zTrain = transpose(PCsTrain) # PCsTrain = (T, k) on doit transposer
    print("Transposition")

elseif (sizePCs[2] == T) 
    zTrain = PCsTrain # PCsTrain = (k, T) pas besoin de transposer
    print("Données OK")
else 
    error("Incohérence entre PCsTrain et tTrain : size(PCsTrain) = 
    $(sizePCs), length(tTrain) = $(T)")
end
k, tCheck = size(zTrain)
@assert tCheck == T "Nombre de pas de temps incohérent après construction de zTrain"
println("zTrain size (k, T): ", size(zTrain))

PCsTrain size = (2922, 150) and length(tTrain) = 2922
TranspositionzTrain size (k, T): (150, 2922)


## Création de nos matrices 

In [3]:
z = zTrain[:, 1:end-1] # (k, T-1)
zP = zTrain[:, 2:end] # 

print("Z size : ", size(z))
print("Zp size : ", size(zP))

Z size : (150, 2921)Zp size : (150, 2921)

## Calcul de l'opérateur de Koopman

In [4]:
zPseudoInv = pinv(z) # Pseudo Inverse de Z : (2921, 150)
A =zP * zPseudoInv

println("A size : ", size(A))

A size : (150, 150)


## Analyse de A

In [5]:
spect = eigen(A) # décomposition spectrale
valPropres = spect.values # valeurs propres
vectPropres = spect.vectors # vecteurs propres

println("Nombre de valeurs propres = ", length(valPropres))

Nombre de valeurs propres = 150


## Prédiction avec Koopman (baseline)

In [6]:
#using Plots 
using Statistics

tValues = length(tVal)
sizeValues = size(PCsVal)
println("PCsVal size = ",sizeValues, " and length(tValues) = ", tValues)

zValues = nothing 

if sizeValues[1] == tValues
    zValues = transpose(PCsVal) # on transpose
elseif sizeValues[2] == tValues
    zValues = PCsVal
else
    error("Incohérence entre PCsVal et tVal")
end
println("zValues size (k, tValues) = ", size(zValues))

zValuesPresent = zValues[:, 1:end-1] # états en t
zValuesFuture = zValues[:, 2:end] # états en t+1

zValuesPredicted = A * zValuesPresent
error1Step = zValuesPredicted - zValuesFuture
rmse1Step = sqrt(mean(error1Step .^ 2))

println("\nRMSE one step Koopman sur la validation = ", rmse1Step)

# Multi step 

function koopmanRoll(A::AbstractMatrix, z0::AbstractVector, numberSteps::Int)
    tailleVect = length(z0)
    trajectoire = Matrix{eltype(A)}(undef, tailleVect, numberSteps + 1)
    trajectoire[:, 1] = z0
    z = copy(z0)
    for n in 1:numberSteps
        z = A * z 
        trajectoire[:, n+1] = z
    end 
    return trajectoire # (k, numberSteps + 1)
end 

# Evaluation sur toute notre période


z0Val = zValues[:, 1] # état initial
numberStepsVal = size(zValues, 2) - 1 # nombre de pas à prédire

trajectoirePredieVal = koopmanRoll(A, z0Val, numberStepsVal) # (k, tValues)

# Erreur à chaque instant
erreurTrajectoire = trajectoirePredieVal - zValues
# Vecteur de RMSE dans le temps
rmseTrajectoire = sqrt.(mean(erreurTrajectoire .^ 2; dims = 1))[:]

println("RMSE multi step moyen sur la validation = ", mean(rmseTrajectoire))

# Diagnostic

println("\n Diagnostics rapides : ")
println("\n RMSE oneStep = ", rmse1Step)
println("\n RMSE multiStep moy = ", mean(rmseTrajectoire))
### Affichage RMSE sur les 5 premiers pas de temps
println("\n RMSE multiStep (5 premiers pas) = ")
println(rmseTrajectoire[1:min(5, length(rmseTrajectoire))])

### Comparaison mode EOF (premier par exemple) réel contre prédiction sur les 5 premiers temps
modIndex = 1
println("\n Mode EOF numéro ", modIndex," réel contre prédit (5 premiers temps) : ")
println("\n Réel = ", zValues[modIndex, 1:min(5, size(zValues, 2))])
println("\n Prédit = ", trajectoirePredieVal[modIndex, 1:min(5, size(trajectoirePredieVal, 2))])

PCsVal size = (730, 150) and length(tValues) = 730
zValues size (k, tValues) = (150, 730)



RMSE one step Koopman sur la validation = 0.961402
RMSE multi step moyen sur la validation = 1.1296895

 Diagnostics rapides : 

 RMSE oneStep = 0.961402

 RMSE multiStep moy = 1.1296895

 RMSE multiStep (5 premiers pas) = 
Float32[0.0, 0.5881988, 1.2120327, 0.99952126, 0.9884687]

 Mode EOF numéro 1 réel contre prédit (5 premiers temps) : 

 Réel = Float32[0.71609473, 0.6209254, 0.4138982, 0.55332565, 0.6048181]

 Prédit = Float32[0.71609473, 0.6845454, 0.6485234, 0.61538213, 0.58562994]
